## Аналіз A/B тесту

Маємо проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це класична гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку. У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми проаналізуємо вплив на утримання (retention) гравців. Тобто хочемо зрозуміти чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Змінні в даних наступні:

- userid - унікальний номер, який ідентифікує кожного гравця.
- version - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- sum_gamerounds - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- retention_1 - чи через 1 день після встановлення гравець повернувся і почав грати?
- retention_7 - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

ФОРМУЛЮЄМО ГІПОТЕЗУ:

H₀ (нульова гіпотеза): Переміщення воріт не вплинуло на утримання гравців на 7 день.
H₁ (альтернативна гіпотеза): Переміщення воріт вплинуло на утримання гравців на 7 день.

In [1]:
#імпорт необхідних бібліотек

import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

%matplotlib inline

In [3]:
import os
os.getcwd()

'/Users/mariaslovinska/Desktop/data lovers/Домашка статистики'

In [6]:
file_path = "data_statistics/cookie_cats.csv"

In [9]:
df = pd.read_csv(file_path)

In [13]:
df.shape

(90189, 5)

In [15]:
df.head(10)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
5,540,gate_40,187,True,True
6,1066,gate_30,0,False,False
7,1444,gate_40,2,False,False
8,1574,gate_40,108,True,True
9,1587,gate_40,153,True,False


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [30]:
df['version'].value_counts()# Щоб переконатися, що розподіл рівномірний.

version
gate_40    45489
gate_30    44700
Name: count, dtype: int64

розподіл майже рівномірний

In [42]:
sample_difference = round(100 - (44700 / 45489) * 100, 2)
sample_difference

1.73

Група gate_30 на 1.73% менша за gate_40.

Різниця в розмірах груп невелика (~1.73%), тому дисбаланс незначний, і A/B-тест залишається коректним.

Упевнимося, що немає користувачів, які були обрані кілька разів.

In [45]:
session_counts = df['userid'].value_counts(ascending=False)
multi_users = session_counts[session_counts > 1].count()

print(f'Є {multi_users} користувачів, які зустрічаються кілька разів у наборі даних.')

Є 0 користувачів, які зустрічаються кілька разів у наборі даних.


In [48]:
df.groupby('version')['retention_7'].mean()

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64

зробимо як в лабораторній:

In [53]:
retention7_rates = df.groupby('version')['retention_7']


In [57]:
std_dev = lambda x: np.std(x, ddof=0)  # Standard deviation sigma (σ)
sem = lambda x: stats.sem(x)           # Standard error of the mean (SEM)

retention7_rates = retention7_rates.agg([np.mean, std_dev, sem])
retention7_rates.columns = ['retention7_rate', 'std_deviation', 'std_error']

retention7_rates.style.format('{:.3f}')

/var/folders/81/2ddflf315k3c6pyg5gm3rfs80000gn/T/ipykernel_19355/1565720805.py:4: FutureWarning: The provided callable <function mean at 0x107c59120> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  retention7_rates = retention7_rates.agg([np.mean, std_dev, sem])


,retention7_rate,std_deviation,std_error
version,,,
gate_30,0.190,0.392,0.002
gate_40,0.182,0.386,0.002


retention на 7-й день вище у групи gate_30, тобто перенесення воріт на рівень 40 могло негативно вплинути на повернення гравців.

2. Перевірте з допомогою z-тесту аналогічно до прикладу в лекції, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для двох вибірок. Виведіть результат у форматі:
```
z statistic: ...
p-value: ...
Довірчий інтервал 95% для групи control: [..., ...]
Довірчий інтервал 95% для групи treatment: [..., ...]
```
де замість `...` - обчислені значення. В якості висновка дайте відповідь на два питання:  
    1. чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
    2. чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  
    
Зверніть увагу, в такому і схожому завданнях ми використовуєм `proportion` Z-тест. Це тому що в нас залежна змінна має бінарне значення (повернеться аби ні користувач, чи клікне або ні користувач в інших ситуаціях - всього два можливих значення в змінної: 0/1, True/False ). Якщо б ми вимірювали скажімо чи є стат. значущою різниця між вагою чоловіків і жінок в певній вибірці, ми б використовувавли функцію `statsmodels.stats.ztest`, бо залежна змінна `вага` є неперервною (тип float, замість типу int чи bool і тільки двох можливих значень).

In [59]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [67]:
gate_30_results = df[df['version'] == 'gate_30']['retention_7'] ## oбираємо дані для обох груп
gate_40_results = df[df['version'] == 'gate_40']['retention_7']

# Кількість гравців у кожній групі
n_gate_30 = gate_30_results.count()
n_gate_40 = gate_40_results.count()

# Кількість гравців, які повернулися через 7 днів
successes = [gate_30_results.sum(), gate_40_results.sum()]
nobs = [n_gate_30, n_gate_40]

# Виконуємо Z-тест
z_stat, pval = proportions_ztest(successes, nobs=nobs)

# Обчислюємо 95% довірчі інтервали для кожної групи
(lower_30, lower_40), (upper_30, upper_40) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'Z-статистика: {z_stat:.2f}')
print(f'P-значення: {pval:.3f}')
print(f'Довірчий інтервал 95% для gate_30: [{lower_30:.3f}, {upper_30:.3f}]')
print(f'Довірчий інтервал 95% для gate_40: [{lower_40:.3f}, {upper_40:.3f}]')

# Перевіряємо рівень значущості
alpha = 0.05
if pval < alpha:
    print("Є статистично значуща різниця між групами!")
else:
    print("Немає статистично значущої різниці між групами.")


Z-статистика: 3.16
P-значення: 0.002
Довірчий інтервал 95% для gate_30: [0.187, 0.194]
Довірчий інтервал 95% для gate_40: [0.178, 0.186]
Є статистично значуща різниця між групами!


Довірчі інтервали для gate_30 та gate_40 не перетинаються. Якщо межі інтервалів не перетинаються, це означає, що різниця між групами статистично значуща — різниця в результатах не є випадковою.
Також ми бачимо, що p-value значно менше рівня значущості 0.05, тому якщо воно менше - ми відхиляємо нульову гіпотезу.
Висновок: зміна розташування воріт ґ чутливою для показника утримання гравців-клієнтів. Варто продовжити спостереження.

3. Є ще один тип тестів, який використовується для бінарної метрики як от "зробить юзер дію, чи ні" - тест [**Хі-квадрат**](https://www.bmj.com/about-bmj/resources-readers/publications/statistics-square-one/8-chi-squared-tests) (ще одне [пояснення](https://www.scribbr.com/statistics/chi-square-tests/) тесту з прикладами). В нього інші гіпотези Н0 і Н1 на відміну від z- та t-тестів. А також цей тест можна використовувати, якщо в нас більше за 2 досліджувані групи, тобто в нас не А/В тест, а А/B/C/D, наприклад.  

В **z- та t-тестах** (які відрізняються тим, що ми в першому не знаємо дисперсію генеральної сукупності, але якщо в нас великий набір даних, то ці два тести дають дуже схожі результати) **ми перевіряємо, чи є різниця у середніх показниках по групам користувачів**.  

А в **тесті Хі-квадрат ми перевіряємо чи є звʼязок між групою користувача і тим, чи він зробить цікаву нам дію**. Це ніби дослідження одного і того самого, але дещо різними способами. Для перевірки, можна виконувати кілька тестів (особливо, якщо один дає якийсь непереконливий результат типу р-значення 0.07 - наче і fail to regect H0 на рівні стат значущості 5%, але цікаво, що скажуть інші тести), тож, зробимо і ми тест хі-квадрат та порівняємо його результат з z-тестом.

Про різницю між тестами можна почитати ще [тут](https://stats.stackexchange.com/a/178860) - це просто пояснення користувача стековерфлоу, але там розумні люди сидять.

Для проведення хі-квадрат тесту скористаємось функцією з `scipy.stats` `chi2_contingency` для обчислення статистики хі-квадрат і р-значення для перевірки конкретної гіпотези. У цю функцію вам треба передати таблицю 2х2: кількість випадків для кожної версії гри і значення `retention_7`.

**Задача**: виконайте тест хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та тим, чи зайде гравець на 7ий день після встановлення гри.
Тут гіпотези наступні
- Н0: значення retention_7 не залежить від версії гри
- Н1: є залежність між версією гри і значенням retention_7

Виведіть p-значення та зробіть висновок.


Для виконання тесту хі-квадрат потрібно виконати кілька кроків:

Створити таблицю спряженості (contingency table): Це таблиця 2x2, яка відображає зв'язок між двома категоріями (наприклад, версією гри та тим, чи зайшов гравець на 7-й день).

Застосувати функцію chi2_contingency з бібліотеки scipy.stats для обчислення статистики хі-квадрат і p-значення.

Проаналізувати p-значення для прийняття або відхилення гіпотез.

In [72]:
contingency_table = pd.crosstab(df['version'], df['retention_7']) ## Створимо таблицю спряженості 2x2
contingency_table


retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [76]:
from scipy.stats import chi2_contingency

In [80]:
# Виконуємо тест хі-квадрат
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

# Виводимо результат
print(f'p-значення: {p_value:.3f}')

# Висновок
if p_value < 0.05:
    print("Є статистично значуща залежність між версією гри та retention на 7ий день.")
else:
    print("Немає статистично значущої залежності між версією гри та retention на 7ий день.")


p-значення: 0.002
Є статистично значуща залежність між версією гри та retention на 7ий день.
